### Information Retreival - LangChain
- Without Using OpenAI Embeddings
- Without OpenAI LLM

Two Applications:
- Text Documents
- Multiple PDF Files

In [10]:
# !pip install langchain
# !pip install huggingface_hub
# !pip3 install sentence_transformers

### Get HUGGINGFACEHUB_API_KEY

In [11]:
import os
import json

with open("config.json", "r+") as f : 
    config = json.load(f)
f.close()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = config['API_Keys']['huggingFace_token']

### Download Text File

In [ ]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [ ]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [6]:
# documents

In [7]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [8]:
# print(wrap_text_preserve_newlines(str(documents[0])))

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

42

In [ ]:
docs[0]

Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.', metadata={'source':

### Embeddings

In [12]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [9]:
!pip3 install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 20.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [13]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS
# db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "What did the president say about the Supreme Court"
docs = db.similarity_search(query)

In [ ]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And
while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an
Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer,
thank you for your service.

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the
United States Supreme Court.

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our
nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### Create QA Chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub


In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":512})

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "What did the president say about the Supreme Court"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court'

In [ ]:
query = "What did the president say about economy?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Build the economy from the bottom up and the middle out, not from the top down'

'One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court'

'not enough information'

### Working with PDF Files

In [14]:
!pip3 install unstructured
# !pip3 install chromadb
!pip3 install Cython
!pip3 install tiktoken
!pip3 install unstructured[local-inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 47.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 49.0 MB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 48.9 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━

In [15]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [16]:
pdf_paths = "/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/" 
pdfs = [os.path.join(pdf_paths, f) for f in os.listdir(pdf_paths) if f.endswith(".pdf")]
pdfs.sort()
pdfs

['/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase0.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase1.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase2.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase3.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase4.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase5.pdf']

In [17]:
loaders = [UnstructuredPDFLoader(pdfs[3])]
loaders

In [20]:
!pip3 install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 34.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [21]:
from langchain.text_splitter import CharacterTextSplitter

index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)

[nltk_data] Downloading package punkt to /Users/nandy/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nandy/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
Using embedded DuckDB without persistence: data will be transient


In [32]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

# llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0.9, "max_length":64})
llm=HuggingFaceHub(repo_id="stabilityai/stablelm-tuned-alpha-3b", model_kwargs={"temperature":0.9, "max_length":64})


In [33]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")

In [34]:
chain.run('What percentage of carbon dioxide consumption causes headaches?')

'\n'

In [ ]:
chain.run('Who are the authors of GPT4all technical report?')

'Yuvanesh Anand'

In [ ]:
chain.run('What is the model size of GPT4all?')

'quantized 4-bit versions of the model allowing virtually anyone to run the model on CPU'